In [245]:
import sqlite3
import re
import csv

In [3]:
base_name = 'data.sqlite' #название вашей базы
file_name = 'ex_text.txt' #путь к вашему файлу
table_name = 'Example' #название вашей таблицы
page = '13a' #первая часть адреса первой строки в вашем файле
num_line = 1 #вторая часть адреса первой строки в вашем файле

In [476]:
base_name = 'tonmon.sqlite'
con = sqlite3.connect(base_name)
cur = con.cursor()

In [477]:
def create_table(table_name, pr_base=None, pr_table=None, pr_file=None):
    cur.execute('''CREATE TABLE IF NOT EXISTS {table_name}
    (ID INTEGER, ADDRESS text, KOR text, TRANSLIT text, BASE text, SUF text, GLOSS text, AUTO_GLOSS text, TRANS text, ContKOR text, RUS text, WClass text, COMMENT text, PRIMARY KEY (ID  AUTOINCREMENT))'''.format(table_name=table_name))
    data = None
    if pr_base is not None:
        con2 = sqlite3.connect(pr_base)
        cur2 = con2.cursor()
        cur2.execute('''SELECT ADDRESS, KOR, ОСНОВА, ХВОСТ, GLOSS, TRANS, ContKOR, RUS, WClass, Поле1 FROM {pr_table}'''.format(pr_table=pr_table))
        data = cur2.fetchall()
        con2.commit()
        con2.close()
    if pr_file is not None:
        data = []
        with open(pr_file) as f:
            if pr_file[-3:] == 'tsv':
                dl = '\t'
            else:
                dl = ','
            for row in csv.reader(f, delimiter=dl):
                if pr_file not in ['Nastya2.tsv', 'Nastya3.tsv']:
                    data.append(row[1:])
                else:
                    data.append(row)
        data = data[1:]
        if 'Denis' in pr_file or 'Nastya' in pr_file:
            data = [rw[:2] + rw[3:7] + rw[8:6:-1] + rw[2:3] + rw[-1:] for rw in data]
        elif 'Seva' in pr_file:
            data = [rw[:2] + rw[3:7] + [''] + rw[7:8] + rw[2:3] + rw[-1:] for rw in data]
        print(data[0], len(data))
    if data is not None:
        cur.executemany('''INSERT INTO {table_name} ('ADDRESS', 'KOR', 'BASE', 'SUF', 'GLOSS',
        'TRANS', 'ContKOR', 'RUS', 'WClass', 'COMMENT') VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''.format(table_name=table_name),
                        data)
    con.commit()

In [478]:
pr_base = 'data.sqlite'
pr_tables = ['Denis4', 'Dasha5','Nastya6', 'GenConc']
for pr_tab in pr_tables:
    create_table('ver_B', pr_base=pr_base, pr_table=pr_tab)

In [312]:
pr_files = ['Dasha1.csv', 'Nastya1.tsv', 'Seva1.csv', 'Denis1.csv', 'Dasha2.csv', 'Nastya2.tsv', 'Seva2.csv', 'Denis2.csv', 'Dasha3.csv', 'Nastya3.tsv']
for pr_f in pr_files:
    print(pr_f)
    create_table('ver_C', pr_file=pr_f)

Dasha1.csv
['1b 1', '하\ue288과', 'ha-nol', 'kwa', 'COMIT', 'Heaven ', '', '', 'N', ''] 83
Nastya1.tsv
['3a 1', '아비와', 'a-pi', 'wa', 'COMIT', 'father', '', '', 'N', ''] 107
Seva1.csv
['4b', '님금과', 'nimkum', 'kwa', 'COMIT', 'ruler', '', '임금?', 'N', ''] 117
Denis1.csv
['6b1', '지아비와', 'ci-a-pi', 'wa', 'COMI', 'husband', '', '', 'N', ''] 148
Dasha2.csv
['8b 1', '얼운과', 'el-wun', 'kwa', 'COMI', 'senior', '어른', '', 'N', ''] 155
Nastya2.tsv
['10b 1', '벗과', 'pes', 'kwa', 'COMIT', 'friend', '', '', 'N', ''] 144
Seva2.csv
['14a', '이', 'i', '', '', 'this', '', '', 'DET', ''] 137
Denis2.csv
['15a15', '그', 'ku', '', '', 'that', '', '', 'PN', ''] 149
Dasha3.csv
['15b 11', '이러모로\ue73b', 'i-le', 'mulwo=psey', 'INSTR', 'this way', '', '', 'N?', ''] 89
Nastya3.tsv
['24a 8', '연옥에셔', 'yen-wok', 'ey-sye', 'LOC', 'prison', '', '', 'N', ''] 233


In [473]:
def get_voc(table_name):
    cur.execute('SELECT DISTINCT SUF, GLOSS, WClass FROM {} GROUP BY SUF, WClass'.format(table_name))
    voc = cur.fetchall()
    con.commit()
    v_dic = {}
    for suf, gloss, wc in voc:
        if suf and suf != '' and gloss and wc:
            sufs = re.split('=|\+', suf)
            glosses = re.split('\+|=', gloss)
            for s, gl in zip(sufs, glosses):
                if wc == 'BV' or 'NEGPrep' in wc or 'COP' in wc:
                    wc = 'P'
                if wc == 'BN':
                    wc = 'N'
                if (s, wc) not in v_dic:
                    v_dic[(s, wc)] = [gl]
                elif gl not in v_dic[(s, wc)]:
                    v_dic[(s, wc)].append(gl)
                if gl == 'NMNZ' or wc == 'BN':
                    wc = 'N'
                if gl == 'COP':
                    wc = 'P'
                    
    return v_dic

In [354]:
cur.execute('SELECT DISTINCT GLOSS FROM ver_B')
gs = cur.fetchall()
cur.execute('SELECT DISTINCT GLOSS FROM ver_C')
gs2 = cur.fetchall()
gls = set()
for i in gs + gs2:
    if i[0] and i[0] != '':
        for w in re.split(r'\+|=|-', i[0]):
            gls.add(w.upper())

In [355]:
#gl_dic = {'DECL': ' DECL', 'DECL', 'DECK'}
gl_dic = {}
for g in gls:
    if len(g.strip(' ')) > 4:
        if g.strip(' ')[:4] not in gl_dic:
            gl_dic[g.strip(' ')[:4]] = [g]
        else:
            gl_dic[g.strip(' ')[:4]].append(g)
    else:
        if g not in gl_dic:
            gl_dic[g] = [g]
        else:
            gl_dic[g].append(g)

In [449]:
gl_dic['HONSub'] = ['HONSUBJ', 'HONSUB']
gl_dic['HONSub.VOL'] = ['HONSUB.VOL']
gl_dic['COMIT'] = ['COMI?', 'COMI', 'COMIT']
gl_dic['ADVS'] = ['ADV(S)', 'ADVS', 'ADV']
gl_dic['?'] = ['?', '??', '???']
gl_dic['LOC'] = ['LOC?', 'LOC']
gl_dic['DECL'] = ['DECL/QUOT', 'DECL', 'DECK']
gl_dic['NMNZ'] = ['NOMZ', '?NOMZ', 'NMLZ', 'NMNZ']
gl_dic['TOP'] = ['TOP', '?TOP']
gl_dic['INTER'] = ['INTER', 'INTERWH', 'INTR']
gl_dic['QUOT'] = ['QOUT', 'QUOT']
gl_dic['LYRDECL'] = ['LYCDECL', 'LYRDEC']
gl_dic['NEG'] = ['NEG?', 'NEG']
gl_dic['CV'] = ['CV', 'CONV', 'CONVERB']
gl_dic['SHOULD'] = ['SHOULD']
gl_dic['IND'] = ['IND?', 'IND']
gl_dic['PProb'] = ['PPROB']
gl_dic['DRV{N}'] = ['DRV{N}']

In [358]:
del gl_dic['NMLZ']
del gl_dic['DRV{']
del gl_dic['NOM(']
del gl_dic['SHOU']
del gl_dic['PPRO']
del gl_dic['IND?']
del gl_dic['NEG?']
del gl_dic['HONS']
del gl_dic['COMI']
del gl_dic['ADV(']
del gl_dic['ADV']
del gl_dic['??']
del gl_dic['???']
del gl_dic['LOC?']
del gl_dic['?TOP']
del gl_dic['?NOM']
del gl_dic['DECK']
del gl_dic['NOMZ']
del gl_dic['INTR']
del gl_dic['QOUT']
del gl_dic['LYCD']
del gl_dic['LYRD']
del gl_dic['INTE']

In [450]:
un_gls = {}
for k, vals in gl_dic.items():
    for val in vals:
        un_gls[val] = k

In [479]:
def uni_gloss(un_gls, table_name):
    cur.execute('SELECT ID, GLOSS FROM {}'.format(table_name))
    vals = cur.fetchall()
    for i, gl in vals:
        if gl is not None and gl != '':
            gl = '+'.join([un_gls[s.upper()] for s in re.split('\+|=|-', gl)])
            cur.execute('UPDATE {} SET GLOSS = ? WHERE ID = {}'.format(table_name, i), (gl,))
            con.commit()

In [482]:
table_name = 'ver_B'
uni_gloss(un_gls, 'ver_B')

In [485]:
def auto_gloss(base_name):
    voc = get_voc(base_name)
    cur.execute('SELECT ID, SUF, WClass FROM {}'.format(table_name))
    vals = cur.fetchall()
    glosses = []
    for i, suf, wc in vals:
        if suf and wc and suf != '':
            sufs = re.split('=|\+', suf)
            if wc == 'BN':
                wc = 'N'
            if wc == 'BV' or 'NEGPrep' in wc or 'COP' in wc:
                wc = 'P'
            gl = []
            for s in sufs:
                if (s, wc) in voc:
                    gl.append('|'.join(voc[(s, wc)]))
                    if voc[(s, wc)] == ['NMNZ']:
                        wc = 'N'
                else:
                    gl.append('?')
            gl = '+'.join(gl)
            cur.execute('UPDATE {} SET AUTO_GLOSS = ? WHERE ID = {}'.format(table_name, i), (gl,))
            con.commit()

In [486]:
auto_gloss('ver_B')

In [4]:
def fill_table(file_name, table_name, page, num_line):
    with open(file_name, 'r') as f:
        for line in f:
            new_page = re.search('\d+[ab]', line)
            if new_page:
                num_line = 1
                address = new_page.group(0) + ' ' + str(num_line)
                page = new_page.group(0)
            else:
                address = page + ' ' + str(num_line)
            words = line.split()
            for word in words:
                if new_page and page in word:
                    word = re.sub(page, '', word)
                if len(word) > 0:
                    c.execute('INSERT INTO {table_name} (ADDRESS, KOR) VALUES (?, ?)'.format(table_name=table_name), (address, word))
            num_line += 1
        
        
    conn.commit()

In [11]:
conn.close()

In [ ]:
#!pip install jamotools
hangul_dict = {'ㄱ': 'k', 'ㄴ': 'n', 'ㄷ': 't', 'ㄹ': 'l', 'ㅁ': 'm', 'ㅂ': 'p', 
               'ㅅ': 's', 'ㅇ': 'ng', 'ㅈ': 'c', 'ㅊ': 'ch', 'ㅋ': 'kh', 'ㅌ':'th',
               'ㅍ': 'ph', 'ㅎ': 'h', 'ㅏ': 'a', 'ㅓ': 'e', 'ㅗ': 'wo', 'ㅜ': 'wu',
               'ㅡ': 'u', 'ㅣ': 'i', 'ㅑ': 'ya', 'ㅕ': 'ye', 'ㅛ': 'ywo', 'ㅠ': 'ywu',
              '\ue97d': 'o', 'ㅿ': 'z', 'ㆁ': '°', 'ㆆ': 'q', 'ㅱ': 'w*', 'ㅸ': 'W', 
               'ㄲ': 'kk', 'ㄸ': 'tt', 'ㅃ': 'pp', 'ㅆ': 'ss', 'ㅉ': 'cc', 'ㅐ': 'ay',
               'ㅒ': 'yay', 'ㅔ': 'ey', 'ㅖ': 'yey', 'ㅘ': 'wa', 'ㅙ': 'way', 'ㅚ': 'woy', 
               'ㅝ': 'we', 'ㅞ': 'wey', 'ㅟ': 'wuy', 'ㅢ': 'uy'}

In [12]:
#!pip install python-magic

You should consider upgrading via the '/home/marynepo/anaconda3/envs/py37/bin/python -m pip install --upgrade pip' command.


In [17]:
#import magic
#blob = open(file_name, 'rb').read()
#m.load()
#encoding = m.buffer('')